# DATASCI W261: Machine Learning at Scale
## Assignment Week 2
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

### HW2.0:

#### What is a race condition in the context of parallel computation? Give an example.
A race condition is when a section of code is executed by multiple processes, and the order in which the processes execute will impact the final result.

![Race condition example](race_conditions.png)
Source: https://en.wikipedia.org/wiki/Race_condition#Example

#### What is MapReduce?
MapReduce can refer to multiple concepts:
- **Programming model:** Processes are split into a "mapping" phase, and a "reducing" phase. In the map phase, a certain function is mapped on to each value in a data set, and then in the reduce phase, the result of the map phase is aggregated. 
- **Execution framework:** This framework coordinates running processes written with the above model in mind.
- **Software implementation:** MapReduce is the name of Google's proprietary implementation of this programming model, while Apache Hadoop is the open-source implementation.

#### How does it differ from Hadoop?
Hadoop is the open-source implementation of Google's MapReduce. Hadoop consists of two parts: distributed storage of data, and distributed processing of data. HDFS is the storage part, and MapReduce is the processing part.

#### Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.
Hadoop is based on the MapReduce paradigm. The classic example of the MapReduce programming paradigm is word count. In the map phase of word count, each word in a document is assigned a count of 1. In the reduce phase, the counts for each unique word are summed to yield the final count of each word.

In [20]:
def hw2_0():
    doc = "Hello this is a test to test word count test should have a count of three".lower()
    key_vals = []
    
    print "MAP PHASE"
    for word in doc.split():
        print [word, 1]
        key_vals.append([word, 1])
    
    print "\nREDUCE PHASE"
    key_vals = sorted(key_vals)
    
    current_word = None
    current_count = 0
    
    for pair in key_vals:
        if current_word == pair[0]:
            print [current_word, current_count], "(intermediate step)"
            current_count += pair[1]
        else:
            if current_word:
                print [current_word, current_count], "FINAL SUM"
            current_word = pair[0]
            current_count = pair[1]

    print [current_word, current_count], "FINAL SUM"
    
hw2_0()

MAP PHASE
['hello', 1]
['this', 1]
['is', 1]
['a', 1]
['test', 1]
['to', 1]
['test', 1]
['word', 1]
['count', 1]
['test', 1]
['should', 1]
['have', 1]
['a', 1]
['count', 1]
['of', 1]
['three', 1]

REDUCE PHASE
['a', 1] (intermediate step)
['a', 2] FINAL SUM
['count', 1] (intermediate step)
['count', 2] FINAL SUM
['have', 1] FINAL SUM
['hello', 1] FINAL SUM
['is', 1] FINAL SUM
['of', 1] FINAL SUM
['should', 1] FINAL SUM
['test', 1] (intermediate step)
['test', 2] (intermediate step)
['test', 3] FINAL SUM
['this', 1] FINAL SUM
['three', 1] FINAL SUM
['to', 1] FINAL SUM
['word', 1] FINAL SUM


### HW2.1: Sort in Hadoop MapReduce
**Given as input: Records of the form `<integer, “NA”>`, where integer is any integer, and “NA” is just the empty string.**<br>
**Output: Sorted key value pairs of the form `<integer, “NA”>`; what happens if you have multiple reducers? Do you need additional steps? Explain.**

If there are multiple reducers, then a straightforward MapReduce process will yield outputs that are sorted within each reducer, but not sorted across all reducers. In order to output a sort across all reducers, an extra step needs to be implemented that will intelligently send keys to reducers so that the result from all reducers will yield a complete sort. For example, let's say our keys ranged from 0-300. If we had 3 reducers, we could send all keys in the range [0,100) to reducer 1, [100, 200) to reducer 2, and [200, 300] to reducer 3. Thus, the output of each reducer will yield documents that are completely sorted. We would need to balance the keys sent to each reducer to ensure that the load is still balanced between all reducers, which will require some calculations.

#### Write code to generate N  random records of the form `<integer, “NA”>`. Let N = 10,000.

In [28]:
import random

with open("random.txt", "w") as myfile:
    for i in range(10000):
        myfile.write("{:d},{:s}\n".format(random.randint(0, 100000), "NA"))

#### Write the Python Hadoop streaming map-reduce job to perform this sort.

In [29]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Miki Seltzer
## Description: mapper code for HW2.1

import sys

# Our input comes from STDIN (standard input)
for line in sys.stdin:
    # In this case, we do not need to map the input to anything
    print line.strip()

Writing mapper.py


In [30]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Miki Seltzer
## Description: reducer code for HW2.1

from operator import itemgetter
import sys

for line in sys.stdin:
    # In this case, we do not need to reduce anything
    print line.strip()

Writing reducer.py


In [31]:
!chmod +x mapper.py
!chmod +x reducer.py

'chmod' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
